In [1]:
import os
import cv2
from tqdm import tqdm
from random import shuffle
import numpy as np
from PIL import Image as img1
from PIL import ImageEnhance as imgEn
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
TEST_DIR = "F:/DR/Disease Grading/Original Images/Testing Set/"


DATA_SAVE_DIR_TEST = "F:/DR/Disease Grading/Original Images/Test_UNCONT/"
CONT_DATA_SAVE_DIR_TEST = "F:/DR/Disease Grading/Original Images/Test_CONT/"



In [3]:
def contrast_crop(PATH):
    pre_temp = (0,0,0)
    for img in tqdm(os.listdir(PATH)):
        orig_image_path = os.path.join(PATH,img)
        cont_image_path = contrast_(PATH,img)
#         print(orig_image_path)
        imo = img1.open(str(orig_image_path))
        contrast = imgEn.Contrast(imo)
        imc=contrast.enhance(2)
        
        size_x, size_y = imc.size
        pixel = imc.load()
        #LEFT TRAVERSAL
        for i in range(size_x):
            temp = pixel[i,(size_y/2)]
            if pre_temp != temp:
                x_left = i
                break
        #RIGHT TRAVERSAL    
        for i in range(size_x-1,0,-1):
            temp = pixel[i,(size_y/2)]
            if pre_temp != temp:
                x_right = i
                break
        #UP TRAVERSAL
        for i in range(size_y):
            temp = pixel[(size_x/2),i]
            if pre_temp != temp:
                y_top = i
                break
        #DOWN TRAVERSAL    
        for i in range(size_y-1,0,-1):
            temp = pixel[(size_x/2),i]
            if pre_temp != temp:
                y_bott = i
                break
                
        area = ( x_left, y_top, x_right, y_bott)
        imo = imo.crop(area)
        
        imc = img1.open(str(cont_image_path))
        imc = imc.crop(area)
        
        imo.save(DATA_SAVE_DIR_TEST+img, "JPEG", quality=80, optimize=True, progressive=True)
        imc.save(CONT_DATA_SAVE_DIR_TEST+img, "JPEG", quality=80, optimize=True, progressive=True)
        
        

In [4]:
def contrast_(PATH,img_name):
    #print(PATH,img_name)
    bgr = cv2.imread(PATH+img_name)
    #print(bgr.shape)
    lab = cv2.cvtColor(bgr,cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(10,10))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    cv2.imwrite(CONT_DATA_SAVE_DIR_TEST + img_name,bgr)
    return CONT_DATA_SAVE_DIR_TEST + img_name
    
    

In [5]:
contrast_crop(TEST_DIR)

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [02:38<00:00,  1.51s/it]
